In [1]:
import numpy as np
import math
from scipy.stats import norm 

In [2]:
# Binomial Tree parameters

S = 45         # Current stock price
K = 40         # Option strike price
T = 1.0        # Time to expiration in years
r = 0.065      # Risk-free interest rate
sigma = 0.2    # Volatility of the underlying stock
n = 1000

def binomial_option_price (S, K, T, r, vol, option_type='call', steps=1000):
   
    dt = T / n
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    p = (math.exp(r * dt) - d) / (u - d)

    option_prices = [[0 for _ in range(n + 1)] for _ in range(n + 1)]

    # Calculate option prices at expiration
    for j in range(n + 1):
        if option_type == 'call':
            option_prices[n][j] = max(0, S * (u**j) * (d**(n - j)) - K)
        elif option_type == 'put':
            option_prices[n][j] = max(0, K - S * (u**j) * (d**(n - j)))

    # Calculate option prices at earlier time steps
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            if option_type == 'call':
                option_prices[i][j] = max(0, math.exp(-r * dt) * (p * option_prices[i + 1][j + 1] + (1 - p) * option_prices[i + 1][j]))
            elif option_type == 'put':
                option_prices[i][j] = max(0, math.exp(-r * dt) * (p * option_prices[i + 1][j + 1] + (1 - p) * option_prices[i + 1][j]))

    return option_prices[0][0]



call_price = binomial_option_price(S, K, T, r, sigma, option_type='call')
put_price = binomial_option_price(S, K, T, r, sigma, option_type='put')


print(f"Binomial Call Option Price: {call_price:.2f}")
print(f"Binomial Put Option Price: {put_price:.2f}")

Binomial Call Option Price: 8.32
Binomial Put Option Price: 0.80


In [3]:
#Heston Model using Euler-Maruyama scheme & Monte Carlo to calculate option prices

# Parameters
S0 = 45.0       # Initial stock price
K = 40.0        #Strick Price
V0 = 0.2       # Initial volatility
r = 0.065        # Risk-free rate
kappa = 2.0     # Mean reversion speed
theta = 0.04    # Long-term average volatility
sigma = 0.1     # Volatility of volatility
rho = -0.7      # Correlation between asset price and volatility
T = 1.0         # Time to maturity
N = 252         # Number of time steps per year
M = 1000       # Number of paths

# Function to generate paths using Euler-Maruyama scheme
def heston_mc(S0, V0, r, kappa, theta, sigma, rho, T, N, M):
    dt = T / N
    sqrt_dt = np.sqrt(dt)
    sqrt_V = np.sqrt(V0)
    
    # Initialize arrays for stock prices and volatilities
    S = np.zeros((M, N+1))
    S[:, 0] = S0
    V = np.zeros((M, N+1))
    V[:, 0] = V0
    
    # Generate random variables
    Z_S = np.random.normal(size=(M, N))
    Z_V = rho * Z_S + np.sqrt(1 - rho**2) * np.random.normal(size=(M, N))
    
    for t in range(1, N+1):
        # Stock price simulation (exact solution for Heston model)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * V[:, t-1]) * dt + np.sqrt(V[:, t-1] * dt) * Z_S[:, t-1])
        
        # Volatility simulation (exact solution for Heston model)
        V[:, t] = V[:, t-1] + kappa * (theta - V[:, t-1]) * dt + sigma * np.sqrt(V[:, t-1] * dt) * Z_V[:, t-1]
        V[:, t] = np.maximum(V[:, t], 0)  # Take care of negative volatilities
    
    return S, V

# Generate paths
np.random.seed(42)  # for reproducibility
S, V = heston_mc(S0, V0, r, kappa, theta, sigma, rho, T, N, M)

# Calculate option prices using Monte Carlo simulation
K = 40.0  # Strike price
discount_factor = np.exp(-r * T)
ST = S[:, -1]  # Terminal stock price

# Calculate call option price
call_payoff = np.maximum(ST - K, 0)
call_price = discount_factor * np.mean(call_payoff)

# Calculate put option price
put_payoff = np.maximum(K - ST, 0)
put_price = discount_factor * np.mean(put_payoff)

print(f"Heston Call Option Price: ${call_price:.2f}")
print(f"Heston Put Option Price: ${put_price:.2f}")

Heston Call Option Price: $9.87
Heston Put Option Price: $2.56


In [15]:
#Black Scholes Model 

#Parametres

S = 45.0      # Spot Price 
K = 40.0      # Strike Price 
T = 1.0    # Time of expiration 
r = 0.065   # Risk Free rate 
vol = 0.2   # Volatility 

#Calculating D1 

d1 = (math.log(S/K) +( r + 0.5*vol**2)*T) / (vol*math.sqrt(T))

#Calculating D2 

d2 = d1 - (vol*math.sqrt(T))

#Calculating Call Option Price 

C = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

#Calculating Put Option 

P = K * math.exp(-r * T) * norm.cdf (-d2) - S * norm.cdf(-d1) 


print(' The Value of d1 is: ',  round(d1, 4 )) 
print(' The Value of d2 is: ',  round(d2, 4 ))
print('Black Scholes Call Option is: $', round(C, 2)) 
print('Black Schloes Put Option is: $', round(P, 2)) 

 The Value of d1 is:  1.0139
 The Value of d2 is:  0.8139
Black Scholes Call Option is: $ 8.32
Black Schloes Put Option is: $ 0.8


In [16]:
import numpy as np

#Monte Carlo 

# Parameters
S0 = 100.0  # Current stock price
K = 105.0   # Strike price
r = 0.065    # Risk-free rate
sigma = 0.2 # Volatility
T = 1.0     # Time to expiry (in years)
num_simulations = 1000

# Simulation function
def monte_carlo_option_price(S0, K, r, sigma, T, option_type):
    z = np.random.standard_normal(num_simulations)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    
    if option_type == 'call':
        payoff = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - ST, 0)
    
    option_price = np.exp(-r * T) * np.mean(payoff)
    
    return option_price

# Calculate option prices
call_option_price = monte_carlo_option_price(S0, K, r, sigma, T, 'call')
put_option_price = monte_carlo_option_price(S0, K, r, sigma, T, 'put')

# Print results
print(f"Monte Carlo European Call Option Price: ${call_option_price:.2f}")
print(f"Monte Carlo European Put Option Price: ${put_option_price:.2f}")


Monte Carlo European Call Option Price: $8.48
Monte Carlo European Put Option Price: $7.21


In [17]:
import yfinance as yf

# Step 1: Retrieve stock price data for Rivian Automotive (RIVN) for the last 3 months
ticker = "RIVN"
start_date = "2024-04-01"
end_date = "2024-07-01"
stock_data = yf.download(ticker, start=start_date, end=end_date)

# Extract adjusted closing prices
stock_prices = stock_data['Adj Close']

# Calculate daily returns
daily_returns = stock_prices.pct_change().dropna()

# Step 2: Define parameters for Monte Carlo simulation
# Number of simulations
num_simulations = 1000
# Number of trading days
num_days = len(daily_returns)
# Initial stock price (last adjusted close price)
S0 = stock_prices[-1]
# Risk-free rate (approximated to 6.5% annually)
r = 0.065 / 252
# Volatility (annualized standard deviation of daily returns)
volatility = daily_returns.std() * np.sqrt(252)

# Step 3: Monte Carlo simulation function to estimate option prices
def monte_carlo_option_price(S0, K, r, volatility, T, option_type):
    # Generate random numbers for stock price simulations
    z = np.random.standard_normal(num_simulations)
    # Simulate stock price paths using geometric Brownian motion
    ST = S0 * np.exp((r - 0.5 * volatility**2) * T + volatility * np.sqrt(T) * z)
    
    # Calculate option payoffs at maturity
    if option_type == 'call':
        payoff = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - ST, 0)
    
    # Discounted expected payoff to present value
    option_price = np.exp(-r * T) * np.mean(payoff)
    
    return option_price

# Define parameters for the options
strike_price = S0 * 1.1  # Strike price set to 110% of current stock price
time_to_expiry = 1  # Time to expiration in years (approximated to 1 year)

# Calculate Monte Carlo option prices
call_option_price = monte_carlo_option_price(S0, strike_price, r, volatility, time_to_expiry, 'call')
put_option_price = monte_carlo_option_price(S0, strike_price, r, volatility, time_to_expiry, 'put')

# Print the results
print(f"Monte Carlo Call Option Price: ${call_option_price:.2f}")
print(f"Monte Carlo Put Option Price: ${put_option_price:.2f}")

#Monte Carlo simulation for option pricing based on historical stock data. 
#Adjustments may be needed based on specific requirements or additional features desired (e.g., dividend yield consideration, more sophisticated volatility modeling).



[*********************100%%**********************]  1 of 1 completed

Monte Carlo Call Option Price: $4.35
Monte Carlo Put Option Price: $5.39



C:\Users\DELL-L7480\AppData\Local\Temp\ipykernel_1352\2999481030.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  S0 = stock_prices[-1]
